In [182]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
# import xgboost as xgb
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())


0it [00:00, ?it/s]


In [183]:
train = pd.read_csv('./train_1.csv').fillna(0)
key = pd.read_csv('./key_1.csv')

In [184]:
sub = pd.read_csv("./sample_submission_1.csv")

In [185]:
for col in train.columns[1:]:
    train[col] = pd.to_numeric(train[col],downcast='integer')

In [287]:
def get_median(row, last_n_days):
    return pd.rolling_median(pd.Series(row), window = last_n_days)    

In [288]:
def get_mean(row, last_n_days):
    return pd.rolling_mean(pd.Series(row), window = last_n_days)    

In [292]:
def get_long_stats(func_type, last_n_days, df, df_melt):

    if func_type == "median":
        rolling_stats = df.iloc[:, 1:].progress_apply(get_median, last_n_days = last_n_days, axis = 1)
    elif func_type == "mean":
        rolling_stats = df.iloc[:, 1:].progress_apply(get_mean, last_n_days = last_n_days, axis = 1)

    rolling_stats_df = pd.concat([df.Page, rolling_stats.apply(pd.Series)], axis = 1)    
    rolling_stats_df.columns = df.columns
    
    rolling_stats_melt = pd.melt(rolling_stats_df, id_vars=['Page'], var_name="date", value_name=func_type + "_" + str(last_n_days))
    
    df_melt = pd.merge(df_melt, rolling_stats_melt, on = ["Page", "date"])     
    return df_melt

In [294]:
train_melt = pd.melt(train, id_vars=['Page'], var_name="date", value_name="visits")

train_melt = get_long_stats("mean", 7, train, train_melt)
train_melt = get_long_stats("mean", 30, train, train_melt)
train_melt = get_long_stats("mean", 60, train, train_melt)

train_melt = get_long_stats("median", 7, train, train_melt)
train_melt = get_long_stats("median", 30, train, train_melt)
train_melt = get_long_stats("median", 60, train, train_melt)


  0%|          | 1/145063 [00:00<16:25:59,  2.45it/s]/Library/Python/2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=30,center=False).mean()
  

 21%|██        | 30151/145063 [00:15<00:51, 2235.15it/s]


 43%|████▎     | 62214/145063 [00:29<00:34, 2394.01it/s]


 66%|██████▌   | 95277/145063 [00:44<00:21, 2332.73it/s]


 88%|████████▊ | 127831/145063 [00:58<00:07, 2358.93it/s]


100%|█████████▉| 144877/145063 [01:05<00:00, 2367.26it/s]
145064it [01:09, 2097.25it/s]                            
  0%|          | 1/145063 [00:00<15:55:09,  2.53it/s]/Library/Python/2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=60,center=False).mean()
  

 10%|▉         | 14057/145063 [00:06<00:55, 2381.00it/s]


...

 60%|█████▉    | 87023/145063 [01:23<00:55, 1045.06it/s]


 70%|███████   | 101597/145063 [01:38<00:40, 1072.53it/s]


 80%|███████▉  | 116003/145063 [01:51<00:28, 1024.38it/s]


 90%|████████▉ | 130550/145063 [02:05<00:13, 1084.73it/s]


100%|█████████▉| 144990/145063 [02:19<00:00, 1142.11it/s]
145064it [02:22, 1017.18it/s]                            

In [ ]:
def get_source(row):
    return row.Page.split["."][-1].split("_")[-1]

In [ ]:
def get_language(row):
     return row.Page.split["."][0].split("_")[1]

In [300]:
train_melt.to_csv("train_feats.csv", index = False)

In [86]:
key["trunc_page"] = key.apply(lambda x: x["Page"][:-11], axis = 1)
key.to_csv("key_trunc.csv", index = False)

In [301]:
# train_melt = pd.read_csv("train_feats.csv")
key = pd.read_csv("key_trunc.csv")

train_melt = pd.merge(train_melt, key, left_on="Page", right_on="trunc_page", how="inner")
train_melt.drop("Page_x", inplace=True, axis = 1)
train_melt.drop("Page_y", inplace=True, axis = 1)

train_melt.to_csv("train_merged.csv", index = False)

KeyboardInterrupt: 

In [93]:
train_merged_filtered = train_merged[["Id", "median_60_days"]]

In [111]:
sub = pd.merge(sub, train_merged_filtered, on = "Id")
sub.drop("Visits", inplace=True, axis = 1)
sub.columns = ["Id", "Visits"]

In [112]:
sub.to_csv("median_60_days.csv", index = False)

In [299]:
train_melt

,Page,date,visits,mean_7,mean_30,mean_60,median_7,median_30,median_60
0,2NE1_zh.wikipedia.org_all-access_spider,2015-07-01,18,NaN,NaN,NaN,NaN,NaN,NaN
145063,2NE1_zh.wikipedia.org_all-access_spider,2015-07-02,11,NaN,NaN,NaN,NaN,NaN,NaN
290126,2NE1_zh.wikipedia.org_all-access_spider,2015-07-03,5,NaN,NaN,NaN,NaN,NaN,NaN
435189,2NE1_zh.wikipedia.org_all-access_spider,2015-07-04,13,NaN,NaN,NaN,NaN,NaN,NaN
580252,2NE1_zh.wikipedia.org_all-access_spider,2015-07-05,14,NaN,NaN,NaN,NaN,NaN,NaN
725315,2NE1_zh.wikipedia.org_all-access_spider,2015-07-06,9,NaN,NaN,NaN,NaN,NaN,NaN
870378,2NE1_zh.wikipedia.org_all-access_spider,2015-07-07,9,11.285714,NaN,NaN,11.0,NaN,NaN
1015441,2NE1_zh.wikipedia.org_all-access_spider,2015-07-08,22,11.857143,NaN,NaN,11.0,NaN,NaN
1160504,2NE1_zh.wikipedia.org_all-access_spider,2015-07-09,26,14.000000,NaN,NaN,13.0,NaN,NaN
1305567,2NE1_zh.wikipedia.org_all-access_spider,2015-07-10,24,16.714286,NaN,NaN,14.0,NaN,NaN


In [ ]:
def run_xgb(x_train, x_test, x_label):

    # x_train = pd.concat([pos_train, neg_train]) #Concat positive and negative
    # y_train = (np.zeros(len(pos_train)) + 1).tolist() + np.zeros(len(neg_train)).tolist() #Putting in 1 and 0

    # x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=4242)

    # Set our parameters for xgboost
    params = {}
    params['objective'] = 'binary:logistic'
    params['eval_metric'] = 'logloss'
    params['eta'] = 0.05
    params['max_depth'] = 6
    params['silent'] = 1

    d_train = xgb.DMatrix(x_train, label=x_label)
    d_test = xgb.DMatrix(x_test)

    watchlist = [(d_train, 'train')]

    bst = xgb.train(params, d_train, 100, watchlist, early_stopping_rounds=50, verbose_eval=50)

    p_test = bst.predict(d_test)

    # xgb.plot_importance(bst)
    # pyplot.show()

    return p_test